# Social Network Analysis - Exercise Sheet 4a)


### Roles in graphs.

In this exercise, we will determine roles in graphs based on (equivalence) relations. More specifically, you will implement methods which decide whether some relation is fullfilled. Based on these and some generic equivalence relations that a user can specify, you will compute partitions of graphs as well as maximal roles.

#### Guidelines
* Submit your code zipped via [moodle](https://moodle.uni-kassel.de/course/view.php?id=11038) until 12.01.2023 23:55 MEZ
* Use the [NetworkX](https://networkx.github.io/documentation/stable/) library for your graphs.

##### Exercise 1:
1. Implement a function which decides whether two nodes of a graph are structurally equivalent.
2. Implement a function which for some relation decides whether it is an equivalence relation.
3. Implement a function which takes as its parameters some input equivalence decision function and a graph. The function then computes a partition of the graph based on the equivalence relation.
4. Implement a function which, for some input equivalence decision function, computes maximal roles.
5. Provide an example network and compute all roles. What insights do you get?